In [1]:
# Import path configuration
from pathlib import Path
import sys
import logging
from dotenv import load_dotenv

from langchain_core.messages import (
    AIMessage,
    AnyMessage,
    HumanMessage,
    SystemMessage,
    ToolMessage,
)
load_dotenv()
logger  = logging.getLogger(__name__)

def _find_project_root(start: Path) -> Path:
    for parent in [start, *start.parents]:
        if (parent/"pyproject.toml").exists():
            return parent
    raise RuntimeError("Project root not found")

ROOT = _find_project_root(Path.cwd())
SRC = ROOT / "src"

if str(SRC) not in sys.path:
    sys.path.insert(0, str(SRC)) 

In [2]:

from app.utils.test import print_something
from app.adapters.langgraph_agent import LangGraphAgent, MODEL
from app.config import configure_logging

In [3]:
configure_logging()

In [ ]:

from langchain_tavily import TavilySearch 
from langchain_google_genai import ChatGoogleGenerativeAI
from langgraph.checkpoint.sqlite import SqliteSaver

tool = TavilySearch(max_results=2)
# memory = SqliteSaver.from_conn_string(":memory:")
prompt = """You are a smart research assistant. Use the search engine to look up information. \
You are allowed to make multiple calls (either together or in sequence). \
Only look up information when you are sure of what you want. \
ATTENTION: If you need to look up some information before asking a follow up question, you are allowed to do that! \
you have access to tools the tavily search tool amongs them.
IMPORTANT: Make sure to review the tools available to you before making a judgement. \
Also, plan the order of tool calls and reasoning before all else.
"""
model = ChatGoogleGenerativeAI(model=MODEL)
messages = [HumanMessage(content="What is the weather in San Francisco?")]
thread = {"configurable": {"thread_id": "1"}}

In [12]:

with SqliteSaver.from_conn_string(":memory:") as memory:
    abot = LangGraphAgent(model, [tool], system=prompt, checkpointer=memory)
    abot.query_stream("What is the last report on weather in Los Angeles?")


2026-01-21 17:23:25,628::DEBUG::httpcore.connection::close.started
2026-01-21 17:23:25,629::DEBUG::httpcore.connection::close.complete
2026-01-21 17:23:25,630::DEBUG::httpcore.connection::connect_tcp.started host='generativelanguage.googleapis.com' port=443 local_address=None timeout=None socket_options=None


2026-01-21 17:23:25,671::DEBUG::httpcore.connection::connect_tcp.complete return_value=<httpcore._backends.sync.SyncStream object at 0x7f9805e9bf50>
2026-01-21 17:23:25,671::DEBUG::httpcore.connection::start_tls.started ssl_context=<ssl.SSLContext object at 0x7f98071ee720> server_hostname='generativelanguage.googleapis.com' timeout=None
2026-01-21 17:23:25,713::DEBUG::httpcore.connection::start_tls.complete return_value=<httpcore._backends.sync.SyncStream object at 0x7f9805e736d0>
2026-01-21 17:23:25,714::DEBUG::httpcore.http11::send_request_headers.started request=<Request [b'POST']>
2026-01-21 17:23:25,716::DEBUG::httpcore.http11::send_request_headers.complete
2026-01-21 17:23:25,717::DEBUG::httpcore.http11::send_request_body.started request=<Request [b'POST']>
2026-01-21 17:23:25,718::DEBUG::httpcore.http11::send_request_body.complete
2026-01-21 17:23:25,719::DEBUG::httpcore.http11::receive_response_headers.started request=<Request [b'POST']>
2026-01-21 17:23:27,972::DEBUG::httpcore